# Selenium para descargar base de dato de la intranet de Opción de Energía

## Librerías

In [14]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
import pandas as pd

## Page 1 - Login Opcion Comercializadora Intranet

In [2]:
driver = webdriver.Chrome("../Programas/chromedriver")

In [3]:
url = "https://canalagentes.opcionenergia.com/authentication/login"

In [4]:
driver.get(url)

In [5]:
driver.find_element_by_name("user").send_keys("vivolt")
driver.find_element_by_name("password").send_keys("temporal01vivolt")
driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)

## Page 2 - Download "SIPS"

Para asignar la variable busco por el xpath, y desde tools de chrome, abro el codigo html y copio el xpath. 

Pulsando el boton de SIPS:

In [6]:
sips = driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li')

In [7]:
sips.click()

rellenando el CUPS: # agregar funcion para rellenar el cups desde la API

In [8]:
driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")

In [9]:
mostrar_grafico_sips = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button')

In [10]:
mostrar_grafico_sips.click()

In [11]:
descargar_csv = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button')

In [12]:
descargar_csv.click()

## Funciones para el proceso

In [35]:
from config import login_data

ModuleNotFoundError: No module named 'config'

In [105]:
%run config.py

Exception: File `'config.py'` not found.

In [83]:
def init():
    driver = webdriver.Chrome("../Programas/chromedriver")

In [86]:
def intranet(url):
    driver.get(url)

In [62]:
def log_in(self, user_data):
    driver.find_element_by_name("user").send_keys("vivolt")
    driver.find_element_by_name("password").send_keys("temporal01vivolt")
    sleep(0.3)
    driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)
    

In [64]:
def download_consumo(self, cups):
    driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li').click()
    sleep(0.2)
    driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button').click()
    sleep(0.2)
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button').click()

## Modularizar Función 

In [90]:
class Download_Consumo_Cliente:
    url = "https://canalagentes.opcionenergia.com/authentication/login"
    
    def __init__(self, chromedriver, login_data):
        self.driver = chromedriver
        self.login_data = login_data
        
    def intranet(self, url):
        self.driver.get(self.url)
        sleep(0.5)
        
    def log_in(self, login_data):
        driver.find_element_by_name("user").send_keys(self.login_data.get('name'))
        driver.find_element_by_name("password").send_keys(self.login_data.get('password'))
        sleep(0.2)
        driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)
    
    def download_consumo(self, client_cups):
        driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li').click()
        sleep(0.2)
        driver.find_element_by_name("Cups").send_keys(self.client_cups.get('cups'))
        driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button').click()
        sleep(0.2)
        driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button').click()

In [ ]:
cc = Download_Consumo_Cliente(driver, login_data)

In [ ]:
cc.__init__

In [ ]:
cc.intranet

In [ ]:
cc.log_in

In [ ]:
cc.download_consumo

## LLamar BD_Consumo_Cliente con una función

In [ ]:
def get_ConsumoCliente():
    cc = Download_Consumo_Cliente(driver, login_data)
    cc.__init__
    cc.intranet
    cc.log_in
    cc.download_consumo

** El documento se guarda en downloads. No todas los archivos csv son iguales, aunque si de headings. creo

## Calling CSV to DF

In [20]:
Consumo_Cliente_df = pd.read_csv("../Data/csv/ejemplo_SIPS_55555.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2


## Cleaning DF

## Saving DF to CSV in Final_Proyect_Vivolt/Data/CSV